In [ ]:
! pip install neptune-client==0.9.1 --quiet

In [ ]:
!pip install pytorch_lightning==1.2.10 --quiet

In [ ]:
import numpy as np
from sklearn.utils import shuffle
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import neptune.new as neptune
import pytorch_lightning
from pytorch_lightning import LightningModule, Trainer

In [ ]:
run = neptune.init(project='d230640/Aula10', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmZTg1OGU3Yi1jOWE1LTRjMjEtYTJjNS1hMjMwM2Y3NDRjOTUifQ==')

https://app.neptune.ai/d230640/Aula10/e/AUL5-12


In [ ]:
batch_size =  40#@param {type:"integer"}
sentence_size =  1000 #@param {type: "integer"}
learning_rate = 1e-2#@param {type: "number"}
max_epochs = 100#@param {type: "integer"}
batch_size = 40#@param {type: "integer"}

# Define hyperparameters
hparams = {"learning_rate": learning_rate,
          "max_epochs": max_epochs,
          "batch_size": batch_size,
          "sentence_size":sentence_size,

}
hparams

{'batch_size': 40,
 'learning_rate': 0.01,
 'max_epochs': 100,
 'sentence_size': 1000}

In [ ]:
!wget -nc http://files.fast.ai/data/examples/imdb_sample.tgz
!tar -xzf imdb_sample.tgz

File ‘imdb_sample.tgz’ already there; not retrieving.



In [ ]:
import pandas as pd
df = pd.read_csv('imdb_sample/texts.csv')
df.shape
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [ ]:
split_rate = 0.8

treino = df[df['is_valid'] == False]
test = df[df['is_valid'] == True]

df = shuffle(treino)

train = treino[0:int(split_rate*treino.shape[0])]
valid = treino[int(split_rate*treino.shape[0]):]

print('treino.shape:', train.shape)
print('test.shape:', valid.shape)
print('test.shape:', test.shape)

treino.shape: (640, 3)
test.shape: (160, 3)
test.shape: (200, 3)


In [ ]:
X_train = train['text']
Y_train = train['label']
X_valid = valid['text']
Y_valid = valid['label']
X_test = test['text']
Y_test = test['label']

print('X_train.head():', X_train.head())
print('Y_train.head():', Y_train.head())

X_train.head(): 0    Un-bleeping-believable! Meg Ryan doesn't even ...
1    This is a extremely well-made film. The acting...
2    Every once in a long while a movie will come a...
3    Name just says it all. I watched this movie wi...
4    This movie succeeds at being one of the most u...
Name: text, dtype: object
Y_train.head(): 0    negative
1    positive
2    negative
3    positive
4    negative
Name: label, dtype: object


In [ ]:
mapeamento = {'positive': True, 'negative': False}
Y_treino_bool = Y_train.map(mapeamento)
Y_valid_bool = Y_valid.map(mapeamento)
Y_test_bool = Y_test.map(mapeamento)

print(Y_treino_bool.head())

0    False
1     True
2    False
3     True
4    False
Name: label, dtype: bool


# Carregando word embeddings com torchtext

Alternativamente, podemos carregar os word embeddings (mais especificamente, o glove) usando o torchtext.

In [ ]:
!wget -nc http://nlp.stanford.edu/data/glove.6B.zip
!unzip -o glove.6B.zip -d glove_dir

File ‘glove.6B.zip’ already there; not retrieving.

Archive:  glove.6B.zip
  inflating: glove_dir/glove.6B.50d.txt  
  inflating: glove_dir/glove.6B.100d.txt  
  inflating: glove_dir/glove.6B.200d.txt  
  inflating: glove_dir/glove.6B.300d.txt  


In [ ]:
from torchtext.vocab import GloVe
glove_vectors = GloVe(name='6B', dim=300, cache='./glove_dir')
glove_vectors.vectors = torch.vstack((glove_vectors.vectors, glove_vectors.vectors.mean(axis=0, keepdims=True), torch.zeros((1,glove_vectors.vectors.shape[1]))))


In [ ]:
print(glove_vectors.vectors.shape)
print('Primeiras 20 palavras e seus índices:', list(glove_vectors.stoi.items())[:20])

torch.Size([400002, 300])
Primeiras 20 palavras e seus índices: [('the', 0), (',', 1), ('.', 2), ('of', 3), ('to', 4), ('and', 5), ('in', 6), ('a', 7), ('"', 8), ("'s", 9), ('for', 10), ('-', 11), ('that', 12), ('on', 13), ('is', 14), ('was', 15), ('said', 16), ('with', 17), ('he', 18), ('as', 19)]


In [ ]:
glove_vectors.vectors[glove_vectors.stoi['this']]

tensor([-2.0437e-01,  1.6431e-01,  4.1794e-02, -1.3708e-01, -2.9779e-01,
         3.3440e-01, -6.9955e-02, -6.8036e-02,  1.0604e-01, -2.0337e+00,
         1.7977e-01, -7.7403e-02, -1.9518e-01,  1.8324e-01,  3.0017e-02,
        -5.4762e-02, -4.5725e-01, -2.4509e-02,  5.7387e-02, -3.4878e-01,
         3.9696e-02,  4.4826e-01, -5.8462e-02,  4.1181e-01, -3.5411e-02,
        -1.4722e-01,  1.0740e-01, -2.5896e-01, -1.1658e-01,  1.9822e-01,
         3.2850e-01,  2.4177e-01, -5.7177e-01, -5.6442e-02, -9.6437e-01,
         3.4482e-01,  5.4639e-02,  2.3828e-01, -1.9139e-01,  3.0899e-01,
         2.8044e-01, -3.3814e-02, -2.5436e-01,  1.5373e-02,  1.6341e-01,
         2.6352e-01,  1.5812e-01,  3.2044e-01, -2.3082e-01,  2.6050e-01,
         2.0606e-01, -8.9769e-02, -1.0055e-01,  7.0378e-02, -2.7452e-02,
         2.7959e-01, -9.5862e-02,  2.0574e-01,  2.9522e-01, -1.2285e-02,
         1.1164e-01, -5.1373e-02,  4.6106e-01,  2.3014e-02, -3.7141e-01,
        -2.4166e-01,  3.3773e-02,  3.6827e-02,  1.6

In [ ]:
def tokenizer(sentence,vocabulary):
    tokens = []
    for word in sentence.split():
        if word in vocabulary.keys():
            tokens.append(vocabulary[word])
    return tokens

In [ ]:
tokenizer("this is a man amanhãã",glove_vectors.stoi)

[37, 14, 7, 300]

In [ ]:
class DatasetTokenized(Dataset):
    def __init__(self,x,y,vocabulary,sentence_size):
        self.x = x
        self.y = y
        self.sentence_size = sentence_size
        self.vocabulary = vocabulary

    def __len__(self):
        return len(self.y)

    def __getitem__(self,idx):
        x_tokenized = tokenizer(self.x[idx],self.vocabulary)
        if len(x_tokenized) > self.sentence_size:
            x_tokenized = x_tokenized[0:sentence_size]
        elif len(x_tokenized) < self.sentence_size:
            # x_tokenized = x_tokenized + [torch.Tensor(np.zeros(len(self.vocabulary)))]*(sentence_size - len(x_tokenized))
            x_tokenized = x_tokenized + [len(self.vocabulary)]*(sentence_size - len(x_tokenized))
        return torch.Tensor(x_tokenized), int(self.y[idx])

In [ ]:
train_dataset = DatasetTokenized(list(X_train),list(Y_treino_bool),glove_vectors.stoi,sentence_size)
valid_dataset = DatasetTokenized(list(X_valid),list(Y_valid_bool),glove_vectors.stoi,sentence_size)
test_dataset = DatasetTokenized(list(X_test),list(Y_test_bool),glove_vectors.stoi,sentence_size)

train_dataloader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
valid_dataloader = DataLoader(dataset = valid_dataset, batch_size = batch_size, shuffle = False)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)

In [ ]:
x,y = next(iter(train_dataset))
print(x,y)

tensor([1.5100e+02, 6.6200e+02, 7.1000e+01, 3.5180e+03, 8.6321e+04, 3.1906e+04,
        1.4960e+03, 6.0000e+00, 4.2000e+01, 3.5200e+03, 9.0700e+02, 2.8500e+02,
        1.3339e+04, 7.1000e+01, 8.9660e+03, 1.7072e+05, 2.0500e+03, 4.0000e+00,
        7.3300e+02, 6.7000e+01, 1.5000e+01, 0.0000e+00, 1.9380e+03, 1.3000e+01,
        3.7000e+01, 1.0200e+02, 9.2100e+02, 3.0000e+00, 1.5230e+03, 1.3050e+03,
        3.1000e+01, 2.6000e+01, 4.3200e+02, 5.1000e+01, 3.7000e+01, 1.5000e+01,
        1.6810e+03, 2.1000e+01, 0.0000e+00, 1.8560e+03, 3.8000e+01, 1.1900e+02,
        3.0000e+01, 7.0000e+00, 1.1962e+04, 3.0000e+00, 1.1000e+01, 2.2900e+03,
        4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05,
        4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05,
        4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05,
        4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05, 4.0000e+05,
        4.0000e+05, 4.0000e+05, 4.0000e+

In [ ]:
class MyLightningModule(LightningModule):
    def __init__(self, hparams, run, modelo):
        super().__init__()
        print(hparams)
        self.hparams = hparams
        run['hparams'] = hparams  # Log to Neptune.
        self.save_hyperparameters(hparams)
        self.criterion = torch.nn.CrossEntropyLoss(reduction='none')
        self.model = modelo

    def forward(self, x):
        logits = self.model(x)
        preds = logits.argmax(dim=1)
        return logits, preds

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        # predict da rede
        logits = self.model(x)
        # calcula a perda
        batch_losses = self.criterion(logits, y)
        loss = batch_losses.mean()
        run['train/batch_loss'].log(loss)

        # O PL sempre espera um retorno nomeado 'loss' da training_step.
        return {'loss': loss, 'batch_losses': batch_losses}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()

        run['train/loss'].log(avg_loss)
        return
  
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)

        self.log('valid_loss', avg_loss.item(), prog_bar=True)
        self.log('valid_accuracy', accuracy.item(), prog_bar=True)
  
    def test_step(self, val_batch, batch_idx):
        # A implementação deste método é opcional no Pytorch Lightning.
        x, y = val_batch
        
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def test_epoch_end(self, outputs):
        # A implementação deste método é opcional no Pytorch Lightning.
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)

        self.log('test_loss', avg_loss.item())
        self.log('test_accuracy', accuracy.item())

    def configure_optimizers(self):
        # Gradiente descendente
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.hparams['learning_rate'])
        # Aqui usamos um scheduler dummy pois o pytorch lightning original requer um.
        scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 1.0)
        return [optimizer], [scheduler]  # Forma de retorno para associar um otimizador a um scheduler.

In [ ]:
glove_vectors.vectors.shape

torch.Size([400002, 300])

Forma de loop (L=3):

    # ----
    seq = [w1, w2, w3]
    E = []  # new attention embeddings.
    for q in seq:
        scores = []
        for k in seq:
            score = matmul(C(q), C(k)T)
            scores.append(score)
        probs = softmax(scores)

        new_embedding = 0
        for v, p in zip(seq, probs):
            new_embedding += C(v) * p
        E.append(new_embedding)


Forma matricial:
        #----
        X = stack(C(w1), C(w2), C(w3))
        X.shape = B, L, D
        B = tamanho do minibatch
        L = comprimento da sequência
        D = dimensão do embeddings
        Q = K = V = X
        def attention(Q, K, V):
            scores = matmul(Q, KT) # shape = B,L,L
            probs = softmax(scores, dim=-1) # B,L,L
            E = matmul(probs, V)  # shape = B,L,D
            return E 

In [ ]:
#https://stackoverflow.com/questions/58630101/using-torch-nn-embedding-for-glove-should-we-fine-tune-the-embeddings-or-just-u
class EmbeddingNetworkGlove(torch.nn.Module):
    def __init__(self,attention_type):
        super(EmbeddingNetworkGlove, self).__init__()
        self.e = torch.nn.Embedding.from_pretrained(glove_vectors.vectors ,freeze=True)
        self.l1 = torch.nn.Linear(glove_vectors.vectors.shape[1], 100)
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(100, 2)
        self.attention_type = attention_type

    def loop_auto_attention(self, embeddings):
        final_E = []
        for seq in embeddings:
            E = [] # new attention embeddings
            for q in seq:
                scores = []
                for k in seq:
                    score = np.matmul(q.cpu(), k.T.cpu())
                    scores.append(score)
                probs = torch.nn.Softmax(scores)
                new_embedding = 0
                for v, p in zip(seq, probs.dim):
                    new_embedding += v * p
                E.append(new_embedding)
            E_mean = torch.stack(E).mean(dim=0)
            final_E.append(E_mean)
        return torch.stack(final_E)
    
    def matrix_auto_attention(self,embedding): 
        softmax_function = torch.nn.Softmax()
        B, L, D = embedding.shape        
        Q = K = V = embedding.view(B,-1,L,D).view(-1, D)
        scores = torch.matmul(Q, K.T)
        probs = softmax_function(scores)
        E = torch.matmul(probs, V)
        return E.reshape(B,L,D).mean(dim=1)

    def forward(self, x):
        x = self.e(x.long())
        if attention_type == 'loop':
            x = self.loop_auto_attention(x)
        else:
            x = self.matrix_auto_attention(x)
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        return x

In [ ]:
attention_type = 'loop'
net = EmbeddingNetworkGlove(attention_type)

pl_model = MyLightningModule(hparams=hparams,run=run,modelo=net)
trainer = Trainer(max_epochs=hparams['max_epochs'],gpus=1)
%time trainer.fit(pl_model,train_dataloader,valid_dataloader)

GPU available: True, used: True


{'learning_rate': 0.01, 'max_epochs': 20, 'batch_size': 20, 'emb_dim': 300, 'sentence_size': 50}


TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                  | Params
----------------------------------------------------
0 | criterion | CrossEntropyLoss      | 0     
1 | model     | EmbeddingNetworkGlove | 120 M 
----------------------------------------------------
30.3 K    Trainable params
120 M     Non-trainable params
120 M     Total params
480.124   Total estimated model params size (MB)



CPU times: user 43min 44s, sys: 18.5 s, total: 44min 2s
Wall time: 44min 15s


1

In [ ]:
%time trainer.test(pl_model,train_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.7437500357627869, 'test_loss': 0.5330280661582947}
--------------------------------------------------------------------------------
CPU times: user 1min 42s, sys: 269 ms, total: 1min 42s
Wall time: 1min 42s


[{'test_accuracy': 0.7437500357627869, 'test_loss': 0.5330280661582947}]

In [ ]:
%time trainer.test(pl_model,valid_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6187500357627869, 'test_loss': 0.7512950301170349}
--------------------------------------------------------------------------------
CPU times: user 25.6 s, sys: 64.1 ms, total: 25.7 s
Wall time: 25.6 s


[{'test_accuracy': 0.6187500357627869, 'test_loss': 0.7512950301170349}]

In [ ]:
%time trainer.test(pl_model, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6599999666213989, 'test_loss': 0.7263122200965881}
--------------------------------------------------------------------------------
CPU times: user 32 s, sys: 77 ms, total: 32.1 s
Wall time: 32.1 s


[{'test_accuracy': 0.6599999666213989, 'test_loss': 0.7263122200965881}]

In [ ]:
attention_type = 'matrix'
net = EmbeddingNetworkGlove(attention_type)

pl_model = MyLightningModule(hparams=hparams,run=run,modelo=net)
trainer = Trainer(max_epochs=hparams['max_epochs'],gpus=1)
%time trainer.fit(pl_model,train_dataloader,valid_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                  | Params
----------------------------------------------------
0 | criterion | CrossEntropyLoss      | 0     
1 | model     | EmbeddingNetworkGlove | 120 M 
----------------------------------------------------
30.3 K    Trainable params
120 M     Non-trainable params
120 M     Total params
480.124   Total estimated model params size (MB)


{'learning_rate': 0.01, 'max_epochs': 100, 'batch_size': 40, 'sentence_size': 1000}



CPU times: user 2min 44s, sys: 27.7 s, total: 3min 12s
Wall time: 19min 29s


1

In [ ]:
%time trainer.test(pl_model,train_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.792187511920929, 'test_loss': 0.4081706702709198}
--------------------------------------------------------------------------------
CPU times: user 600 ms, sys: 18.1 ms, total: 618 ms
Wall time: 1.19 s


[{'test_accuracy': 0.792187511920929, 'test_loss': 0.4081706702709198}]

In [ ]:
%time trainer.test(pl_model,valid_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.6875, 'test_loss': 0.8349726796150208}
--------------------------------------------------------------------------------
CPU times: user 130 ms, sys: 2.8 ms, total: 133 ms
Wall time: 129 ms


[{'test_accuracy': 0.6875, 'test_loss': 0.8349726796150208}]

In [ ]:
%time trainer.test(pl_model, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.75, 'test_loss': 0.604928731918335}
--------------------------------------------------------------------------------
CPU times: user 163 ms, sys: 4.6 ms, total: 167 ms
Wall time: 165 ms


[{'test_accuracy': 0.75, 'test_loss': 0.604928731918335}]